In [ ]:
import os
import sys
import serial
import threading
import datetime
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import re
import csv
import logging

# Google Sheets scope and client initialization
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s', stream=sys.stdout)

# Main GUI Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.geometry("800x480")
        self.root.title("RTFED V.01")

        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.data_queue = queue.Queue()
        self.data_to_save = {}
        self.gspread_client = None

        self.setup_gui()

    def setup_gui(self):
        tk.Label(self.root, text="Your Name:", font=("Helvetica", 8, "bold")).grid(column=0, row=0, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=2, pady=2)

        tk.Label(self.root, text="Experiment Name:", font=("Helvetica", 8, "bold")).grid(column=2, row=0, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.W, padx=2, pady=2)

        tk.Label(self.root, text="Google API JSON File:", font=("Helvetica", 8, "bold")).grid(column=0, row=1, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path)
        self.json_entry.grid(column=1, row=1, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(self.root, text="Browse", command=self.browse_json)
        self.browse_json_button.grid(column=2, row=1, padx=5, pady=10)

        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Helvetica", 8, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id)
        self.spreadsheet_entry.grid(column=1, row=2, sticky=tk.W, padx=2, pady=2)

        self.start_button = tk.Button(self.root, text="START", font=("Helvetica", 10, "bold"), bg="green", fg="white", command=self.start_logging)
        self.start_button.grid(column=1, row=3, padx=2, pady=5)
        self.stop_button = tk.Button(self.root, text="STOP (SAVE & QUIT)", font=("Helvetica", 10, "bold"), bg="red", fg="white", command=self.stop_logging)
        self.stop_button.grid(column=2, row=3, padx=2, pady=5)

        self.browse_button = tk.Button(self.root, text="Browse Data Folder", font=("Helvetica", 10, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=0, row=3, padx=2, pady=5)

        self.canvas = tk.Canvas(self.root, width=100, height=100)
        self.canvas.grid(column=1, row=4, columnspan=2, padx=2, pady=2)

        self.setup_ports()

    def browse_json(self):
        self.json_path.set(filedialog.askopenfilename(title="Select JSON File"))

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.root, text=port_name)
            frame.grid(column=idx, row=6, padx=2, pady=2, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", font=("Helvetica", 8, "italic"), foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=20, height=5)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
            self.port_widgets[port_name] = {'status_label': status_label, 'text_widget': text_widget}

    def start_logging(self):
        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            logging.info("Connected to Google Sheets.")
        except Exception as e:
            messagebox.showerror("Error", f"Error setting up Google Sheets API: {e}")
            return
        self.start_data_collection()

    def start_data_collection(self):
        self.device_mappings = self.get_device_mappings_by_usb_port()
        for mapping in self.device_mappings:
            serial_port, port_identifier = mapping['serial_port'], mapping['port_identifier']
            q = self.port_widgets[port_identifier]['text_widget']
            self.data_to_save[port_identifier] = []
            t = threading.Thread(target=self.read_from_port, args=(serial_port, f"Port_{port_identifier}", q, port_identifier))
            t.daemon = True
            t.start()
            self.threads.append(t)

    def read_from_port(self, serial_port, worksheet_name, text_widget, port_identifier):
        try:
            ser = serial.Serial(serial_port, 115200, timeout=1)
            spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            sheet = self.get_or_create_worksheet(spreadsheet, worksheet_name)
            while not stop_event.is_set():
                data = ser.readline().decode('utf-8').strip()
                data_list = data.split(",")
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                if len(data_list) == len(column_headers) - 1:
                    sheet.append_row([timestamp] + data_list)
                    text_widget.insert(tk.END, f"Data logged: {data_list}\n")
                    text_widget.see(tk.END)
                    self.data_to_save[port_identifier].append([timestamp] + data_list)
        except Exception as e:
            messagebox.showerror("Error", f"Error connecting to port {serial_port}: {e}")

    def stop_logging(self):
        stop_event.set()
        for t in self.threads:
            t.join()
        self.save_all_data()
        self.root.quit()
        self.root.destroy()

    def save_all_data(self):
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name, experiment_name = self.experimenter_name.get().lower().strip(), self.experiment_name.get().lower().strip()
        experiment_folder = os.path.join(self.save_path, experimenter_name, f"{experiment_name}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)
        for port_identifier, data_rows in self.data_to_save.items():
            filename = f"{experiment_folder}/{port_identifier}_{current_time}.csv"
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(column_headers)
                writer.writerows(data_rows)

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
